# YugabyteDB

## Connect to YBA Host

Use ssh to connect to the YB host as a jump box (this will give access to the required ssh keys:

```shell
ssh -i ~/.ssh/gcp.pub uid@platform-url
```

## Connect to Desired DB Node

Use the string provided by the YBA "Connect" action.  E.g.:

```shell
sudo ssh -i /opt/yugabyte/yugaware/data/keys/<customer_id>/<yb_node>_<yb_node_id>-key.pem -ostricthostkeychecking=no -p 54422 yugabyte@<ip_address>
```

## Setup DB Node

Run these commands to quickstart ysqlsh and yb-admin:

`enable_wait_queues=true,enable_deadlock_detection=true,wait_queue_poll_interval_ms=5` don't set these in < 2024.1


In [ ]:
# Initialize secrets from the local .env file here
source .env
echo $MY_SECRET

## Initialise an RF3 Cluster

In [ ]:
yugabyted start --advertise_address=127.0.0.1 --base_dir=/tmp/ybd-gcp1 --cloud_location=gcp.us-east1.us-east1-c --tserver_flags "yb_enable_read_committed_isolation=true,enable_wait_queues=true,enable_deadlock_detection=true,wait_queue_poll_interval_ms=5,ysql_log_statement=all"

In [ ]:
yugabyted start --advertise_address=127.0.0.2 --base_dir=/tmp/ybd-gcp2 --cloud_location=gcp.us-central1.us-central1-b --tserver_flags "yb_enable_read_committed_isolation=true,enable_wait_queues=true,enable_deadlock_detection=true,wait_queue_poll_interval_ms=5,ysql_log_statement=all" --join=127.0.0.1

In [ ]:
yugabyted start --advertise_address=127.0.0.3 --base_dir=/tmp/ybd-gcp3 --cloud_location=gcp.us-west1.us-west1-a --tserver_flags "yb_enable_read_committed_isolation=true,enable_wait_queues=true,enable_deadlock_detection=true,wait_queue_poll_interval_ms=5,ysql_log_statement=all" --join=127.0.0.1

In [ ]:
yugabyted status --base_dir=/tmp/ybd-gcp1

In [ ]:
yugabyted stop --base_dir=/tmp/ybd-gcp3

In [ ]:
yugabyted version

In [ ]:
export PATH=~/tserver/bin:~/master/bin:$PATH
alias ysqlsh="ysqlsh -h /tmp/.yb*"
alias ycqlsh="SSL_CERTFILE=/home/yugabyte/yugabyte-tls-config/ca.crt ycqlsh --ssl `hostname -f` -u cassandra"
alias yba="yb-admin -init_master_addrs `hostname -f` --certs_dir_name=/home/yugabyte/yugabyte-tls-config/"

# Log into YSQL

```sh
ysqlsh
```

## Setup YSQLSH

```sql
\pset pager off
\set ea 'explain (analyze,dist) '
set search_path to <schema>
```

## Insert a Lot of Data

```sql
DO
$$
    DECLARE
        chunk_size INT := 25000;
        total_rows INT := 1000000;
        i          INT := 1;
    BEGIN
        WHILE i <= total_rows
            LOOP
                INSERT INTO yb_user(email, display_name, region, password_hash, avatar_url, full_name)
                SELECT 'foo' || seq || '@bar.com', 'Foo ' || seq, 'USA', '***', '', ''
                from generate_series(i, i + chunk_size - 1) as seq;
                i := i + chunk_size;
                RAISE NOTICE 'Inserted % out of % rows', i, total_rows;
                COMMIT;
            END LOOP;
    END
$$;
```

```sql
insert into test1(upc, status) select floor(random()*100+1)::text, floor(random()*6)::int from generate_series(1,1000);
```

# Insert with pattern UUID generation

```sql
insert into yb_device_tracker (device_id, media_id, status)
    select uuid('cdd7cacd-8e0a-4372-8ceb-'||lpad(seq::text, 12, '0')),
    '48d1c2c2-0d83-43d9-bc17-'||lpad(seq::text, 12, '0'),'ACTIVE'||seq
    from generate_series(1,1000) as seq;
```

```sql
insert into budget_tracker (group_id, interval_start, amount, remaining)
    select uuid('cdd7cacd-8e0a-4372-8ceb-'||lpad(seq::text, 12, '0')), seq%24, 100.00, 99.99
    from generate_series(1,10000) as seq;
```

```sql
insert into user_lists(id, user_id, modified, deleted) select uuid_generate_v4(), 'test'||seq, now(), false from generate_series(2,20000) as seq;
```

# PG Stats Statements

```sql
select substring(regexp_replace(query,E'[\n+]',' ','g'),1,150) as query,calls,rows,total_time from pg_stat_statements order by calls desc;
```

or (from https://dbaclass.com/article/monitor-sql-queries-in-postgres-using-pg_stat_statements/)

```sql
SELECT substring(query, 1, 50) AS query,
round(total_time::numeric, 2) AS total_time,
calls,
round(mean_time::numeric, 2) AS mean,
round((100 * total_time /
sum(total_time::numeric) OVER ())::numeric, 2) AS percentage_cpu
FROM pg_stat_statements
ORDER BY total_time DESC
LIMIT 10;
```

## Helpful Stuff

### > Get the YugabyteDB table properties (can be used on indexes as well):

```sql
select num_tablets from yb_table_properties('payments'::regclass);
```

### > Get Number of Tablets and Size

```sql
select num_tablets, pg_size_pretty(pg_table_size('employees')) from yb_table_properties('employees'::regclass);
```

### > Get Splits

```sql
select yb_get_range_split_clause('payments'::regclass);
```

In [ ]:


curl -s 10.12.16.9:9000/metrics | grep -P --no-group-separator "\"(id|table_id|transactions_running)\"" -A1 | grep -P "\""


#YB Admin

## List CDC Streams (for Xcluster)
```sh
yb-admin -init_master_addrs `hostname -f` --certs_dir_name=/home/yugabyte/yugabyte-tls-config/  list_cdc_streams
```

# YBA API: Check the Status of a YBA Setting (Read/Write Test)

Use the Universe ID 00000000-0000-0000-0000-000000000000 to check the Global setting.

In [ ]:
curl -s --insecure --request GET \
--url https://ycs-gcp-platform-stable01/api/v1/customers/${CUSTOMER_ID}/runtime_config/${UNIVERSE_ID}/key/yb.metrics.db_read_write_test \
--header 'Content-Type: application/json' \
--header "X-AUTH-YW-API-TOKEN: ${AUTH_TOKEN}" | jq

# YBA API: Disable the Read/Write Test for a Given Universe

In [ ]:
curl --insecure --request PUT \
--url https://ycs-gcp-platform-stable01/api/v1/customers/${CUSTOMER_ID}/runtime_config/${UNIVERSE_ID}/key/yb.metrics.db_read_write_test \
--header 'Content-Type: text/plain' \
--header "X-AUTH-YW-API-TOKEN: ${AUTH_TOKEN}" \
--data 'false'

# YBA API: Pause a Universe

In [ ]:
curl --insecure --request POST \
--url https://ycs-gcp-platform-stable01/api/v1/customers/${CUSTOMER_ID}/universes/${UNIVERSE_ID}/pause \
--header 'Content-Type: application/json' \
--header "X-AUTH-YW-API-TOKEN: ${AUTH_TOKEN}" \
--data '{}'

# YBA API: Resume a Universe

In [ ]:
curl --insecure --request POST \
--url https://ycs-gcp-platform-stable01/api/v1/customers/${CUSTOMER_ID}/universes/${UNIVERSE_ID}/resume \
--header 'Content-Type: application/json' \
--header "X-AUTH-YW-API-TOKEN: ${AUTH_TOKEN}" \
--data '{}'

In [ ]:
???

In [ ]:
curl --insecure --request GET \
--url https://ycs-gcp-platform-stable01/api/v1/customers/${CUSTOMER_ID}/universes \
--header 'Content-Type: application/json' \
--header "X-AUTH-YW-API-TOKEN: ${AUTH_TOKEN}" \
--data '{}' | jq

In [ ]:
SOURCE_UNIVERSE_ID="df95841c-6ee4-464f-9ac2-40f545072348"
TARGET_UNIVERSE_ID="43cf7e59-72ba-40cc-a552-016d075f3da4"

function getVersion() {
    local universe="$1"
    echo $(curl -s --insecure --request GET \
                   --url https://ycs-gcp-platform-stable01/api/v1/customers/${CUSTOMER_ID}/universes/${universe} \
                   --header 'Content-Type: application/json' \
                   --header "X-AUTH-YW-API-TOKEN: ${AUTH_TOKEN}" | jq -r .universeDetails.clusters[0].userIntent.ybSoftwareVersion)
}

# get the versions
sourceVersion=$(getVersion $SOURCE_UNIVERSE_ID)
targetVersion=$(getVersion $TARGET_UNIVERSE_ID)

if [[ "$sourceVersion" != "$targetVersion" ]]; then
  echo "Warning: source version $sourceVersion does not match target $targetVersion"
fi

In [ ]:
curl --insecure --request GET \
--url https://ycs-gcp-platform-stable01/api/v1/customers/${CUSTOMER_ID}/dr_configs/6a6c2a2c-a7bb-4b9b-a304-3023c8ece875 \
--header 'Content-Type: application/json' \
--header "X-AUTH-YW-API-TOKEN: ${AUTH_TOKEN}" | jq